In [0]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [0]:
num_encoder_tokens = num_decoder_tokens = 10
max_encoder_seq_length = 16
max_decoder_seq_length = 17

input_data = []
output_data = []

sample_size = 128

# generating random vector data
for _ in range(sample_size):
  sequence_length = np.random.randint(low=3, high=max_encoder_seq_length-1)
  sequence = np.random.randint(low=1, high=num_encoder_tokens, size=sequence_length)

  # using same data for input and output
  input_data.append(np.concatenate((sequence, [0]), axis=0))
  output_data.append(np.concatenate(([0], sequence, [0]), axis=0))

print(input_data[:2])
print(output_data[:2])

encoder_input_data = np.zeros(
    (len(input_data), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_data), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_data), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_sequence, target_sequence) in enumerate(zip(input_data, output_data)):
  for t, char in enumerate(input_sequence):
    encoder_input_data[i, t, char] = 1.
  for t, char in enumerate(target_sequence):
    # decoder_target_data is ahead of decoder_input_data by one timestep
    decoder_input_data[i, t, char] = 1.
    if t > 0:
      # decoder_target_data will be ahead by one timestep
      # and will not include the start character.
      decoder_target_data[i, t - 1, char] = 1.

# encoder_input_data.shape, decoder_input_data.shape, decoder_target_data.shape
decoder_input_data[0][:2], decoder_target_data[0][:2]

In [0]:
# training model
batch_size = 64  # Batch size for training.
epochs = 256  # Number of epochs to train for.
latent_dim = 512  # Latent dimensionality of the encoding space.
num_samples = sample_size  # Number of samples to train on.

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

In [0]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
# reverse_input_char_index = dict(
#     (i, char) for char, i in input_token_index.items())
# reverse_target_char_index = dict(
#     (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
  # Encode the input as state vectors.
  states_value = encoder_model.predict(input_seq)

  # Generate empty target sequence of length 1.
  target_seq = np.zeros((1, 1, num_decoder_tokens))
  # Populate the first character of target sequence with the start character.
  target_seq[0, 0, 0] = 1.

  # Sampling loop for a batch of sequences
  # (to simplify, here we assume a batch of size 1).
  stop_condition = False
  decoded_sentence = []
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict(
        [target_seq] + states_value)

    # Sample a token
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    decoded_sentence.append(sampled_token_index)
#     print(sampled_token_index)
    if len(decoded_sentence) > max_decoder_seq_length:
      stop_condition = True

    # Update the target sequence (of length 1).
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, sampled_token_index] = 1.

    # Update states
    states_value = [h, c]
      
  return decoded_sentence


for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    print("input seq", input_seq.shape)
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_data[seq_index])
    print('Decoded sentence:', decoded_sentence)

In [0]:
from google.colab import files
files.download('s2s.h5')